clean version available at:  
- https://github.com/Personalization-Technologies-Lab/RecSys-Course-HSE-Fall23/tree/main/Seminar3

Installing packages:
```
# polara
pip install --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara
```

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, diags
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns # for better visual aesthetics
sns.set_theme(style='white', context='paper')
%config InlineBackend.figure_format = "svg"

from polara import get_movielens_data

from dataprep import leave_last_out, transform_indices, verify_time_split, reindex_data
from evaluation import topn_recommendations, model_evaluate, downvote_seen_items, calculate_rmse

# Preparing data

In this excercise, we will again work with the Movielens-1M data.  
We will also follow the same "most-recent-item in holdout" strategy for simplicity.  
So the preparation code is the same is in the previous lecture.

In [ ]:
data = get_movielens_data(include_time=True)

In [ ]:
training_, holdout_ = leave_last_out(data, 'userid', 'timestamp')
verify_time_split(training_, holdout_)

In [ ]:
training, data_index = transform_indices(training_, 'userid', 'movieid')
holdout = reindex_data(holdout_, data_index, filter_invalid=True)
holdout = holdout.sort_values('userid')

In [ ]:
data_description = dict(
    users = data_index['users'].name,
    items = data_index['items'].name,
    feedback = 'rating',
    n_users = len(data_index['users']),
    n_items = len(data_index['items']),
    test_users = holdout[data_index['users'].name].values
)
data_description

Let's also explicitly store our testset.

In [ ]:
userid = data_description['users']
seen_idx_mask = training[userid].isin(data_description['test_users'])
testset = training[seen_idx_mask]

# Item-based KNN

Let's focus on item-based KNN.  We define a few convenience functions first.

In [ ]:
def generate_interactions_matrix(data, data_description, rebase_users=False):
    '''
    Convert pandas dataframe with interactions into a sparse matrix.
    Allows reindexing user ids, which help ensure data consistency
    at the scoring stage (assumes user ids are sorted in scoring array).
    '''
    n_users = data_description['n_users']
    n_items = data_description['n_items']
    # get indices of observed data
    user_idx = ... # type your code here
    if rebase_users:
        user_idx, user_index = pd.factorize(user_idx, sort=True)
        n_users = len(user_index)
    item_idx = ... # type your code here
    feedback = ... # type your code here
    # construct rating matrix
    return csr_matrix((feedback, (user_idx, item_idx)), shape=(n_users, n_items))

def cosine_similarity_zd(matrix):
    '''Build cosine similarity matrix with zero diagonal.'''
    similarity = ... # type your code here
    return similarity

And the main implementation starts here.  
If $A$ is a matrix of ratings and  $S$ is an item-similarity matrix ($s_{ij}\in[0, 1]$), then KNN-scores matrix $R$ is computed as:  

- for elementwise weighting:
$$
R=A S^{\top} \oslash\left(B S^{\top}\right),\quad
b_{u i}=\left\{\begin{array}{lr}
1, & \text { if } a_{u i} \text { is known } \\
0 & \text { otherwise }
\end{array}\right.
$$

- for row-wise weighting:
$$
R=AS^\top D_S^{-1},\quad
D_S=\operatorname{diag}(S\mathbf{e})
$$

- for unweighted case:
$$
R=AS^\top
$$

In [ ]:
def build_iknn_model(config, data, data_description):
    # compute similarity matrix
    user_item_mtx = ... # type your code here
    item_similarity = ... # type your code here
    return ... # complete your code with necessary output


def iknn_model_scoring(params, testset, testset_description):
    ...
    user_item_mtx = ... # your code to generate n_test_users x n_items matrix
    return ... # complete your code with necessary output


In [ ]:
iknn_params = ... # your code to build the model

In [ ]:
iknn_scores = ... # your code to gerenerate scores

 ## Evaluation

### rating prediction quality

In [ ]:
calculate_rmse(iknn_scores, holdout, data_description)

### top-n recommendations quality

In [ ]:
... # your code to generate recommendations

In [ ]:
model_evaluate(iknn_recs, holdout, data_description)

### models comparison

In [ ]:
# write code to print metrics of all KNN models with different weightings

<font color=green>

- In your opinion, how the evaluation scores will change if we sample holdout items randomly?

</font>

## Analysis

Let's analyse aggregated statistics for movie ratings.  It may give us hints on performance of KNN  models.

In [ ]:
mode = 'unweighted' # name the weighting mode
recommended_items = pd.Series(iknn_recs.ravel()).value_counts()

In [ ]:
item_ratings = (
    training
    .groupby(data_description['items'])
    [data_description['feedback']]
    .agg(['size', 'mean', 'std'])
)

In [ ]:
item_ratings.loc[recommended_items.head(5).index] # top-5 most frequent recommendations

In [ ]:
ax = item_ratings.plot.scatter(
    'size', 'mean', logx=True, alpha=0.3, figsize=(8, 6),
    title=f'Rating distribution of recommended items, {mode=}'
)
item_ratings.loc[recommended_items.index].plot.scatter(
    'size', 'mean', ax=ax, s=recommended_items*0.05, c='red'
);

In [ ]:
ax = item_ratings.plot.scatter(
    'mean', 'std', s=0.05*item_ratings['size'],
    title=f'Rating deviation of recommended items, {mode=}'
);
item_ratings.loc[recommended_items.index].plot.scatter(
    'mean', 'std', ax=ax, c='red', s=recommended_items*0.05
);

# Neighborhood sampling

In [ ]:
def truncate_similarity(similarity, k):
    '''
    For every row in similarity matrix, pick at most k entities
    with the highest similarity scores. Disregard everything else.
    '''
    similarity = similarity.tocsr()
    inds = similarity.indices
    ptrs = similarity.indptr
    data = similarity.data
    new_ptrs = [0]
    new_inds = []
    new_data = []
    for i in range(len(ptrs)-1):
        start, stop = ptrs[i], ptrs[i+1]
        if start < stop:
            data_ = data[start:stop]
            topk = min(len(data_), k)
            idx = np.argpartition(data_, -topk)[-topk:]
            new_data.append(data_[idx])
            new_inds.append(inds[idx+start])
            new_ptrs.append(new_ptrs[-1]+len(idx))
        else:
            new_ptrs.append(new_ptrs[-1])
    new_data = np.concatenate(new_data)
    new_inds = np.concatenate(new_inds)
    truncated = csr_matrix(
        (new_data, new_inds, new_ptrs),
        shape=similarity.shape
    )
    return truncated

## iKNN with neighborhood sampling

In [ ]:
def build_sampled_iknn_model(config, data, data_description):
    # compute similarity matrix
    user_item_mtx = generate_interactions_matrix(data, data_description)
    item_similarity = truncate_similarity(
        cosine_similarity_zd(user_item_mtx.T),
        config['n_neighbors']
    )
    return item_similarity, config['weighting']

In [ ]:
n_neighbors = 100

iknn_params_uw = build_sampled_iknn_model(
    {'weighting': None, 'n_neighbors': n_neighbors}, training, data_description
)
iknn_params_ew = build_sampled_iknn_model(
    {'weighting': 'elementwise', 'n_neighbors': n_neighbors}, training, data_description
)
iknn_params_rw = build_sampled_iknn_model(
    {'weighting': 'rowwise', 'n_neighbors': n_neighbors}, training, data_description
)

In [ ]:
iknn_scores = iknn_model_scoring(iknn_params, testset, data_description)

### rating prediction quality

In [ ]:
calculate_rmse(iknn_scores, holdout, data_description)

### top-n recommendations quality

In [ ]:
downvote_seen_items(iknn_scores, testset, data_description)
iknn_recs = topn_recommendations(iknn_scores, topn=10)

In [ ]:
model_evaluate(iknn_recs, holdout, data_description)

### Analysis

In [ ]:
mode = ... # name the weighting mode
recommended_items = pd.Series(iknn_recs.ravel()).value_counts()

In [ ]:
ax = item_ratings.plot.scatter(
    'size', 'mean', logx=True, alpha=0.3, figsize=(8, 6),
    title=f'Rating distribution of recommended items, {mode=}'
)
item_ratings.loc[recommended_items.index].plot.scatter(
    'size', 'mean', ax=ax, s=recommended_items*0.05, c='red'
);

In [ ]:
ax = item_ratings.plot.scatter(
    'mean', 'std', s=0.05*item_ratings['size'],
    alpha=0.3, figsize=(8, 6),
    title=f'Rating deviation of recommended items, {mode=}'
);
item_ratings.loc[recommended_items.index].plot.scatter(
    'mean', 'std', ax=ax, c='red', s=recommended_items*0.05
);

<font color=green>  

* Explain why neighborhood sampling changed the picture that way?

</font>

# Asymmetric iKNN with column-wise weighting

- for column-wise weighting:
$$
R=A D_S^{-1}S^\top
$$

In [ ]:
def asy_iknn_model_scoring(params, testset, testset_description):
    item_similarity, weighting = params
    user_item_mtx = generate_interactions_matrix(
        testset, testset_description, rebase_users=True
    )
    # implement column-wise weighting, R = A (S D)^T
    ... # your code to generate scores

    raise ValueError('Unrecognized weighting type')

In [ ]:
iknn_params_cw = build_sampled_iknn_model(
    {'weighting': 'rowwise', 'n_neighbors': n_neighbors}, training, data_description
)
iknn_scores_cw = asy_iknn_model_scoring(iknn_params_cw, testset, data_description)

In [ ]:
downvote_seen_items(iknn_scores_cw, testset, data_description)
iknn_recs_cw = topn_recommendations(iknn_scores_cw)

In [ ]:
model_evaluate(iknn_recs_cw, holdout, data_description)

# User-based KNN

Recall, there's no reason for implementing row-wise weighting scheme in user-based KNN.  
So the options are

- for element-wise weighting:
$$
R=K A \oslash\left(K B\right)
$$


- for unweighted case:
$$
R=KA
$$

where $K$ is a user-similarity matrix. 

Note that the implementation of similarity calculation now has to take into account test users which may be unknown at the build stage.

In [ ]:
def build_uknn_model(config, data, data_description):
    user_item_mtx = generate_interactions_matrix(data, data_description)
    # compute similarity matrix and normalization coefficients
    user_similarity = ...
    weighted = config['weighted']
    return user_item_mtx, user_similarity, weighted

def uknn_model_scoring(params, testset, testset_description):
    user_item_mtx, user_similarity, weighted = params
    test_users = testset_description['test_users']

    if not weighted:
        return ...

    normalizer = ...
    return ...

In [ ]:
uknn_params_uw = build_uknn_model(
    {'weighted': False}, training, data_description
)
uknn_params_ew = build_uknn_model(
    {'weighted': True}, training, data_description
)

In [ ]:
uknn_scores_uw = uknn_model_scoring(uknn_params_uw, None, data_description)
uknn_scores_ew = uknn_model_scoring(uknn_params_ew, None, data_description)

 ## Evaluation

### top-n recommendations quality

In [ ]:
downvote_seen_items(uknn_scores_uw, testset, data_description)
downvote_seen_items(uknn_scores_ew, testset, data_description)

In [ ]:
uknn_recs_uw = topn_recommendations(uknn_scores_uw)
uknn_recs_ew = topn_recommendations(uknn_scores_ew)

In [ ]:
modes = ['unweighted', 'elementwise']
uknn_recs = dict(zip(modes, [uknn_recs_uw, uknn_recs_ew]))


uknn_metrics = {}
for mode, recs in uknn_recs.items():
    uknn_metrics[mode] = metrics = model_evaluate(recs, holdout, data_description)
    print(
        f'Weighting mode: {mode}\n'\
        'HR={:.3}, MRR={:.3}, COV={:.3}\n'.format(*metrics)
    )

### Rating prediction

In [ ]:
uknn_scores = dict(zip(modes, [uknn_scores_uw, uknn_scores_ew]))
uknn_rmse = {}
for mode, scores in uknn_scores.items():
    uknn_rmse[mode] = rmse = calculate_rmse(scores, holdout, data_description)
    print(f'Weighting mode: {mode}\n{rmse=:.3f}\n')

<font color=green>

- Try to explain, why user-based KNN with element-wise weighting provides slightly better RMSE scores than its item-based counterpart?

</font>

# Bulk run helpers

```python
modes = ['unweighted', 'elementwise', 'rowwise']
iknn_recs = dict(zip(modes, [iknn_recs_uw, iknn_recs_ew, iknn_recs_rw]))
iknn_metrics = {}
for mode, recs in iknn_recs.items():
    iknn_metrics[mode] = metrics = model_evaluate(recs, holdout, data_description)
    print(
        f'Weighting mode: {mode}\n'\
        'HR={:.3}, MRR={:.3}, COV={:.3}\n'.format(*metrics)
    )
```

```python
iknn_scores = dict(zip(modes, [iknn_scores_uw, iknn_scores_ew, iknn_scores_rw]))
iknn_rmse = {}
for mode, scores in iknn_scores.items():
    iknn_rmse[mode] = rmse = calculate_rmse(scores, holdout, data_description)
    print(f'Weighting mode: {mode}\n{rmse=:.3f}\n')
```